# Cloud AI Platform Prediction - Custom Prediction Routine V1

In [1]:
import json
import os
import pandas as pd
import time

In [2]:
PREDICT_APP_V1_FOLDER = '../predict_app_v1'
TESTING_DATA_PATH = 'gs://jk-demo-datasets/gasdata/testing.csv'
REGION = "us-central1"
ARTIFACT_BUCKET = "gs://jk-demo-artifacts"

## Create a model resource

In [3]:
MODEL_NAME = "octane_regression"
LABELS = "task=regression,domain=chemometrics"

In [ ]:
!gcloud ai-platform models create  $MODEL_NAME \
--regions=$REGION \
--labels=$LABELS

## Create a model version from a custom prediction routine
### Create a predictor class

In [4]:
os.makedirs(PREDICT_APP_V1_FOLDER, exist_ok=True)

In [5]:
%%writefile $PREDICT_APP_V1_FOLDER/predict.py

import os
import numpy as np

from sklearn.externals import joblib

class OctaneRegressor(object):
    """A custom prediction routine for Octane regressor"""
    
    def __init__(self, model):
        """Stores the model loaded in from_path"""
        self._model = model
        
    def predict(self, instances, **kwargs):
        """Runs inference"""
    
        outputs = self._model.predict(np.asarray(instances))
        
        return outputs.tolist()

        
    @classmethod
    def from_path(cls, model_dir):
        """Loads the model from the joblib file"""
        model_path = os.path.join(model_dir, 'model.joblib')
        model = joblib.load(model_path)
        
        
        return cls(model)
    

Writing ../predict_app_v1/predict.py


### Create a source distribution package

In [6]:
%%writefile $PREDICT_APP_V1_FOLDER/setup.py

from setuptools import setup

setup(
    name='custom-predictor-1',
    description='Custom prediction routine.',
    version='0.1',
    scripts=['predict.py']
)

Writing ../predict_app_v1/setup.py


In [7]:
%cd $PREDICT_APP_V1_FOLDER
%run $PREDICT_APP_V1_FOLDER/setup.py sdist --formats=gztar
%cd -

/home/jupyter/caip-demo/predict_app_v1
running sdist
running egg_info
creating custom_predictor_1.egg-info
writing top-level names to custom_predictor_1.egg-info/top_level.txt
writing dependency_links to custom_predictor_1.egg-info/dependency_links.txt
writing custom_predictor_1.egg-info/PKG-INFO
writing manifest file 'custom_predictor_1.egg-info/SOURCES.txt'
reading manifest file 'custom_predictor_1.egg-info/SOURCES.txt'
writing manifest file 'custom_predictor_1.egg-info/SOURCES.txt'


running check


creating custom-predictor-1-0.1
creating custom-predictor-1-0.1/custom_predictor_1.egg-info
copying files to custom-predictor-1-0.1...
copying predict.py -> custom-predictor-1-0.1
copying setup.py -> custom-predictor-1-0.1
copying custom_predictor_1.egg-info/PKG-INFO -> custom-predictor-1-0.1/custom_predictor_1.egg-info
copying custom_predictor_1.egg-info/SOURCES.txt -> custom-predictor-1-0.1/custom_predictor_1.egg-info
copying custom_predictor_1.egg-info/dependency_links.txt -> custom-predictor-1-0.1/custom_predictor_1.egg-info
copying custom_predictor_1.egg-info/top_level.txt -> custom-predictor-1-0.1/custom_predictor_1.egg-info
Writing custom-predictor-1-0.1/setup.cfg
creating dist
Creating tar archive
removing 'custom-predictor-1-0.1' (and everything under it)
/home/jupyter/caip-demo/notebooks


### Copy the source distribution package to GCS

In [8]:
CUSTOM_ROUTINE_NAME = "custom-predictor-1"
TARBALL_NAME = "custom-predictor-1-0.1.tar.gz"
LOCAL_PATH = "{}/dist/{}".format(PREDICT_APP_V1_FOLDER, TARBALL_NAME)
GCS_PATH = "{}/prediction_routines/{}/{}".format(ARTIFACT_BUCKET, CUSTOM_ROUTINE_NAME, TARBALL_NAME)

In [9]:
!gsutil cp $LOCAL_PATH $GCS_PATH

Copying file://../predict_app_v1/dist/custom-predictor-1-0.1.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  990.0 B/  990.0 B]                                                
Operation completed over 1 objects/990.0 B.                                      


### Create a model version

In [13]:
TRAINED_MODEL_PATH = 'gs://jk-demo-artifacts/models/octane-regressor-r14/'
MODEL_VERSION = 'v02'

In [14]:
!gcloud beta ai-platform versions create $MODEL_VERSION \
--model=$MODEL_NAME \
--origin=$TRAINED_MODEL_PATH \x
--runtime-version=1.14 \
--python-version=3.5 \
--package-uris $GCS_PATH \
--prediction-class predict.OctaneRegressor

Creating version (this might take a few minutes)......done.                    


### Test the model

In [15]:
INPUT_FILE = '../datasets/testing_instances.json'

In [16]:
!gcloud ai-platform predict \
--model $MODEL_NAME \
--version $MODEL_VERSION \
--json-instances $INPUT_FILE

[88.59553605661218, 88.84093111257747, 87.35784874687317, 88.73399177287303, 85.26014664670036, 88.12292248697285]
